In [1]:
import pandas as pd

In [2]:
movement = pd.read_excel('C:\\Users\\decoy\\Documents\\Finance\\transactions.xlsx', sheet_name='Movement')
dividend = pd.read_excel('C:\\Users\\decoy\\Documents\\Finance\\transactions.xlsx', sheet_name='Dividend')

In [3]:
closed_profits = 0

In [4]:
class security:

    security_value = 0

    def __init__(self, date, code, quantity, action, price, fees):
        self.date = date
        self.code = code
        self.quantity = quantity
        self.action = action
        self.price = price
        self.fees = fees
        self.value = 0

        if self.action == 'Sell':
            self.value += self.quantity * self.price - self.fees

        elif self.action == 'Buy':
            self.value -= self.quantity * self.price + self.fees

        security.security_value -= self.value

    def transaction(self, quantity, action, price, fees):
        if self.action == 'Sell':
            self.value += self.quantity * self.price - self.fees

        elif self.action == 'Buy':
            self.value -= self.quantity * self.price + self.fees

        security.security_value -= self.value

    @classmethod
    def from_row(cls, row):
        date, code, quantity, action, price, fees = row
        return cls(date, code, quantity, action, price, fees)

In [5]:
sec_list = []
for index in range(len(movement) - 1, 0, -1):
    name = movement.loc[index]['Code'].replace('.', '_').lower()
    if name not in sec_list:
        sec_list += [name]
        globals()[name] = security.from_row(movement.loc[index])
    else:
        date, code, quantity, action, price, fees = movement.loc[index]
        globals()[name].transaction(quantity, action, price, fees)

    print(name, globals()[name].value)


Date        2020-09-08 00:00:00
Code                    CRO.ASX
Quantity                  49626
Action                      Buy
Price                     0.041
Fees                      14.95
Name: 13, dtype: object
Date        2020-09-04 00:00:00
Code                    CGF.ASX
Quantity                   1006
Action                      Buy
Price                      3.96
Fees                      14.95
Name: 12, dtype: object
Date        2020-08-26 00:00:00
Code                    BRN.ASX
Quantity                  13283
Action                      Buy
Price                       0.3
Fees                      14.95
Name: 10, dtype: object
Date        2020-08-10 00:00:00
Code                    ANZ.ASX
Quantity                    866
Action                     Sell
Price                      18.1
Fees                      19.95
Name: 9, dtype: object
Date        2020-08-03 00:00:00
Code                    ANZ.ASX
Quantity                    866
Action                      Buy
Price    

In [6]:
print(security.security_value)

-87736.1


In [7]:
print(cgf_asx.value)

8541.22
